In [ ]:
from google.colab import drive
drive.mount('/content/drive')

MessageError: Error: credential propagation was unsuccessful

In [ ]:
# Unzip normalized dataset
import os
import shutil

zip_DIR = '/content/drive/MyDrive/tcga_slides_224_zip'
temp_DIR = '/content/temp'

for file in os.listdir(zip_DIR):
    if file.endswith('.zip'):
        zip_file_path = os.path.join(zip_DIR, file)
    else:
        continue

    shutil.unpack_archive(zip_file_path, temp_DIR)

In [ ]:
import pandas as pd
import shutil
import os
from tqdm import tqdm

# Unzip slide tiles and sort them into corresponding folders based on a provided mapping
zip_DIR = '/content/temp/zips'
dataset_DIR = '/content/datasets'

# Load mapping series
label_df = pd.read_csv('liu.csv')
category_map = label_df.set_index('TCGA Participant Barcode')['MSI Status']

# Treat MSS and MSI-L as MSS and MSI-H as MSI
label_map = {
    'MSS': 'MSS',
    'MSI-L': 'MSS',
    'MSI-H': 'MSI'
}
category_map = category_map.map(label_map)

for file in tqdm(os.listdir(zip_DIR)):
    if file.endswith('.zip'):
        zip_file_path = os.path.join(zip_DIR, file)
    else:
        continue

    # Select a category directory to unpack
    patient_id = '-'.join(file.split('-')[:3])

    # Get category from mapping (if exists)
    category = category_map.get(patient_id)
    if category and category in ['MSS', 'MSI']:
        category_dir = os.path.join(dataset_DIR, category)
        os.makedirs(category_dir, exist_ok=True)

        # target_path = os.path.join(category_dir, file[:-4])
        # print(f"Unzipping '{file}' → '{category}/'")
        shutil.unpack_archive(zip_file_path, category_dir)
        os.remove(zip_file_path)
    else:
        print(f"⚠️ No category mapping found for folder '{file}'")

⚠️ No category mapping found for folder 'TCGA-AZ-6601-01Z-00-DX1.40681471-3104-48be-8b57-55dba1f432f8.zip'
⚠️ No category mapping found for folder 'TCGA-FP-A4BE-01Z-00-DX1.813B0887-ED1F-48DD-A2E2-F14EB20D1F71.zip'
⚠️ No category mapping found for folder 'TCGA-4T-AA8H-01Z-00-DX1.A46C759C-74A2-4724-B6B5-DECA0D16E029.zip'
⚠️ No category mapping found for folder 'TCGA-CG-4460-01Z-00-DX1.498845d6-e330-45d7-a5ac-153f2cf9b3c7.zip'
⚠️ No category mapping found for folder 'TCGA-AA-A022-01Z-00-DX1.2673F279-5DF6-4E71-92B2-A589DD8F583B.zip'
⚠️ No category mapping found for folder 'TCGA-BR-A4QL-01Z-00-DX1.90F74AB9-6A84-487B-B992-BC66B71A4E1D.zip'
⚠️ No category mapping found for folder 'TCGA-HJ-7597-01Z-00-DX1.4218e9a0-3ef1-4073-8336-1fe81c0bce90.zip'
⚠️ No category mapping found for folder 'TCGA-NH-A8F7-01Z-00-DX1.5CB8911D-07C3-4EF2-A97D-A62B441CF79E.zip'
⚠️ No category mapping found for folder 'TCGA-AA-A00N-01Z-00-DX1.79D0A833-8411-486B-9BED-7B5E203D02F2.zip'
⚠️ No category mapping found for fold

In [ ]:
import os
import torch
import torchvision.transforms as transforms
from torchvision import datasets
from torch.utils.data import DataLoader

# Ścieżka do katalogu z danymi
DATA_DIR = "datasets"

# Transformacje dla obrazów
transform = transforms.Compose([
    transforms.Resize((224, 224)),  # Skalowanie do AlexNet
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])  # Normalizacja ImageNet
])
#Todo@ Val jest zastosowany
# Tworzenie datasetu ImageFolder
dataset = datasets.ImageFolder(root=DATA_DIR, transform=transform)

# # Podział na zbiór treningowy i walidacyjny (80/20)
# train_size = int(0.8 * len(dataset))
# val_size = len(dataset) - train_size
# train_dataset, val_dataset = torch.utils.data.random_split(dataset, [train_size, val_size])

# DataLoader dla treningu i walidacji
BATCH_SIZE = 256

# train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
# val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False)

# Sprawdzenie etykiet klas
class_names = dataset.classes
print(f"Klasy: {class_names}")  # ['MSS', 'MSIMUT']
# print(f"Liczba próbek: Trening: {len(train_dataset)}, Walidacja: {len(val_dataset)}")

Klasy: ['MSI', 'MSS']


In [ ]:
import torch
import timm
from timm.data import resolve_data_config
from timm.data.transforms_factory import create_transform
from huggingface_hub import login

login()  # login with your User Access Token, found at https://huggingface.co/settings/tokens

# pretrained=True needed to load UNI2-h weights (and download weights for the first time)
timm_kwargs = {
            'img_size': 224,
            'patch_size': 14,
            'depth': 24,
            'num_heads': 24,
            'init_values': 1e-5,
            'embed_dim': 1536,
            'mlp_ratio': 2.66667*2,
            'num_classes': 0,
            'no_embed_class': True,
            'mlp_layer': timm.layers.SwiGLUPacked,
            'act_layer': torch.nn.SiLU,
            'reg_tokens': 8,
            'dynamic_img_size': True
        }
model = timm.create_model("hf-hub:MahmoodLab/UNI2-h", pretrained=True, **timm_kwargs)
transform = create_transform(**resolve_data_config(model.pretrained_cfg, model=model))
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
model.eval()

config.json:   0%|          | 0.00/587 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/2.73G [00:00<?, ?B/s]

VisionTransformer(
  (patch_embed): PatchEmbed(
    (proj): Conv2d(3, 1536, kernel_size=(14, 14), stride=(14, 14))
    (norm): Identity()
  )
  (pos_drop): Dropout(p=0.0, inplace=False)
  (patch_drop): Identity()
  (norm_pre): Identity()
  (blocks): Sequential(
    (0): Block(
      (norm1): LayerNorm((1536,), eps=1e-06, elementwise_affine=True)
      (attn): Attention(
        (qkv): Linear(in_features=1536, out_features=4608, bias=True)
        (q_norm): Identity()
        (k_norm): Identity()
        (attn_drop): Dropout(p=0.0, inplace=False)
        (norm): Identity()
        (proj): Linear(in_features=1536, out_features=1536, bias=True)
        (proj_drop): Dropout(p=0.0, inplace=False)
      )
      (ls1): LayerScale()
      (drop_path1): Identity()
      (norm2): LayerNorm((1536,), eps=1e-06, elementwise_affine=True)
      (mlp): GluMlp(
        (fc1): Linear(in_features=1536, out_features=8192, bias=True)
        (act): SiLU()
        (drop1): Dropout(p=0.0, inplace=False)
    

In [ ]:
# MODIFY

import os
import h5py
from PIL import Image
import torch
from torch.utils.data import Dataset, DataLoader
from tqdm import tqdm
from huggingface_hub import login
import timm
from timm.data import resolve_data_config
from timm.data.transforms_factory import create_transform

# --- CONFIG ---
DATA_DIR = "datasets"            # Root folder: class/patient/img.jpg
HDF5_PATH = "features_uni2h.hdf5"
BATCH_SIZE = 256
NUM_WORKERS = 4

# device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# # --- Load UNI2-h encoder + transform ---
# login()  # authenticated HF access
# timm_kwargs = {
#     'img_size': 224, 'patch_size': 14, 'depth': 24, 'num_heads': 24,
#     'init_values': 1e-5, 'embed_dim': 1536,
#     'mlp_ratio': 2.66667*2, 'num_classes': 0, 'no_embed_class': True,
#     'mlp_layer': timm.layers.SwiGLUPacked, 'act_layer': torch.nn.SiLU,
#     'reg_tokens': 8, 'dynamic_img_size': True,
# }
# model = timm.create_model("hf-hub:MahmoodLab/UNI2-h", pretrained=True, **timm_kwargs)
# transform = create_transform(**resolve_data_config(model.pretrained_cfg, model=model))
# model.eval().to(device)

# --- Dataset & DataLoader ---
class Uni2Dataset(Dataset):
    def __init__(self, root_dir, transform):
        self.samples = []
        for class_name in os.listdir(root_dir):
            cls_path = os.path.join(root_dir, class_name)
            if not os.path.isdir(cls_path): continue
            for patient in os.listdir(cls_path):
                pat_path = os.path.join(cls_path, patient)
                if not os.path.isdir(pat_path): continue
                for image in os.listdir(pat_path):
                    if image.lower().endswith(('.jpg', '.jpeg', '.png', '.bmp')):
                        self.samples.append((os.path.join(pat_path, image),
                                             class_name, patient, image))
        self.transform = transform

    def __len__(self): return len(self.samples)

    def __getitem__(self, idx):
        img_path, class_name, patient, img_name = self.samples[idx]
        mask = Image.open(img_path).convert("RGB")
        img = self.transform(mask)
        return img, class_name, patient, img_name

def collate_fn(batch):
    imgs = torch.stack([b[0] for b in batch])
    class_names = [b[1] for b in batch]
    patient_ids = [b[2] for b in batch]
    image_names = [b[3] for b in batch]
    return imgs, class_names, patient_ids, image_names

dataset = Uni2Dataset(DATA_DIR, transform)
dataloader = DataLoader(dataset, batch_size=BATCH_SIZE,
                        # num_workers=NUM_WORKERS,
                        collate_fn=collate_fn)

# --- Feature Extraction & HDF5 Writing ---
with h5py.File(HDF5_PATH, 'w') as hdf5_file:
    for imgs, class_names, patient_ids, image_names in tqdm(dataloader, desc="UNI2‑h Feature Extraction"):
        imgs = imgs.to(device)
        with torch.no_grad():
            feats = model(imgs)  # shape: [B, 1536]
        feats = feats.cpu().numpy()

        for i in range(feats.shape[0]):
            grp = hdf5_file.require_group(f"{class_names[i]}/{patient_ids[i]}")
            dname = os.path.splitext(image_names[i])[0]
            if dname in grp:
                continue  # skip if already exists
            grp.create_dataset(dname, data=feats[i], compression="gzip")

UNI2‑h Feature Extraction: 100%|██████████| 7955/7955 [13:14:26<00:00,  5.99s/it]


In [ ]:
!mv /content/features_uni2h.hdf5 /content/drive/MyDrive